In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [54]:
path='/content/gdrive/MyDrive/Machine Learning/KNN/film_recommendation/data_after.csv'

data=pd.read_csv(path)
data=data.drop(['Unnamed: 0','popularity','vote_counts'],axis=1)
print(data.head())

                          name  runtime  ...  Western  TVMovie
0                    Toy Story     81.0  ...        0        0
1                      Jumanji    104.0  ...        0        0
2             Grumpier Old Men    101.0  ...        0        0
3            Waiting to Exhale    127.0  ...        0        0
4  Father of the Bride Part II    106.0  ...        0        0

[5 rows x 24 columns]


Okey, I saw some **nan values** in ***runtime*** so I decided to replace them by **zeros**, then I just did some normalizations...

In [55]:
runtime=data['runtime'].fillna(0).to_numpy() # replace nan by 0

def minmax_scaler(runtime_l):
  min=np.min(runtime_l)
  max=np.max(runtime_l)
  runtime_=[(x-min)/(max-min) for x in runtime_l]
  return np.array(runtime_)

runtime_=minmax_scaler(runtime)

# new_runtime = pd.DataFrame({'runtime': runtime})
# data.update(new_runtime)  # didnt work:))

data=data.drop(['runtime'],axis=1)
data['runtime']=runtime_
print(data)

                              name  vote_averages  ...  TVMovie   runtime
0                        Toy Story            7.7  ...        0  0.340336
1                          Jumanji            6.9  ...        0  0.436975
2                 Grumpier Old Men            6.5  ...        0  0.424370
3                Waiting to Exhale            6.1  ...        0  0.533613
4      Father of the Bride Part II            5.7  ...        0  0.445378
..                             ...            ...  ...      ...       ...
995           The Three Caballeros            6.2  ...        0  0.298319
996         The Sword in the Stone            6.9  ...        0  0.331933
997            So Dear to My Heart            6.3  ...        0  0.331933
998  Robin Hood: Prince of Thieves            6.6  ...        0  0.600840
999                   Mary Poppins            7.4  ...        0  0.584034

[1000 rows x 24 columns]


Well, I want to use **sklearn** to handle this task...I'll build my own model later

In [21]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
data_=data.iloc[:,1:]
data_matrix = csr_matrix(data_.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(data_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [39]:
# test with a random sample
test_id = np.random.choice(data.shape[0]) # data 1000*24
print(test_id)

# get movie_name
movie_names=data['name'].to_numpy()

587


In [40]:
distances, indices = model_knn.kneighbors(data_.iloc[test_id,:].values.reshape(1, -1), n_neighbors = 6)

In [57]:
import json

with open('/content/gdrive/MyDrive/Machine Learning/KNN/film_recommendation/genres.json', "r") as f:
  genres_dic=json.load(f)

genres=list(genres_dic.keys())

def decoding(gen_list):
  gen_l=[]
  for i in range(len(genres)):
    if gen_list[i]==1:
      gen_l.append(genres[i])
  return gen_l
    


In [58]:
for i in range(0, len(distances.flatten())):
    if i == 0: # itself
        print('Recommendations for {0} - Genres = {1} - Runtime = {2}: \n'.format(movie_names[test_id], decoding(data_.iloc[test_id,1:-1].to_numpy()), runtime[test_id]))
    else:
        print('{0}: {1} - Genres =  {2} - Runtime = {3}'.format(i, movie_names[indices.flatten()[i]],decoding(data_.iloc[test_id,1:-1].to_numpy()),runtime[test_id]))



Recommendations for Snow White and the Seven Dwarfs - Genres = ['Animation', 'Comedy', 'Family'] - Runtime = 83.0: 

1: The Sword in the Stone - Genres =  ['Animation', 'Comedy', 'Family'] - Runtime = 83.0
2: Welcome to the Dollhouse - Genres =  ['Animation', 'Comedy', 'Family'] - Runtime = 83.0
3: Pinocchio - Genres =  ['Animation', 'Comedy', 'Family'] - Runtime = 83.0
4: Old Yeller - Genres =  ['Animation', 'Comedy', 'Family'] - Runtime = 83.0
5: Blue in the Face - Genres =  ['Animation', 'Comedy', 'Family'] - Runtime = 83.0
